In [18]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta
from ipywidgets import interact, IntSlider
from IPython.display import display
import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
# import seaborn as sns
# from matplotlib import pyplot as plt
# %matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler

from pycaret.classification import * 

# pd.options.display.max_columns = 5000
# pd.options.display.max_rows = 5000

In [191]:
# week_df = pd.read_parquet('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Files for Interactive Prediction Section/dataset_with_prediction_1_week.parquet')
# month_df = pd.read_parquet('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Files for Interactive Prediction Section/dataset_with_prediction_1_month.parquet')
quarter_df = pd.read_parquet('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Files for Interactive Prediction Section/dataset_with_prediction_1_quarter.parquet')
# semester_df = pd.read_parquet('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Files for Interactive Prediction Section/dataset_with_prediction_1_semester.parquet')
# year_df = pd.read_parquet('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Files for Interactive Prediction Section/dataset_with_prediction_1_year.parquet')

In [192]:
def clean_it(weeks, df):
    df.reset_index(inplace=True)
    final_index = df.index.max()
    pos = 'cum_injury_in_'+str(weeks)+'_week'
    for n in range(weeks,0,-1): 
        df.loc[final_index-n+1, pos] = ''
    df.loc[df[pos] == "", 'single_injury_prediction_in_'+str(weeks)+'_week'] = df[pos].shift(1) 
    df.loc[df['single_injury_prediction_in_'+str(weeks)+'_week'] == "", 'single_injury_prediction_in_'+str(weeks)+'_week'] = \
        df['unique_predicted_injuries']
    df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['single_injury_prediction_in_'+str(weeks)+'_week'].cumsum()
    df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_prediction_in_'+str(weeks)+'_week'].fillna('')
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] != "", 'cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_in_'+str(weeks)+'_week']
    
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] != "", 'type_for_'+str(weeks)] = 'Actual Injuries'
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] == "", 'type_for_'+str(weeks)] = 'Predicted Injuries'

    extra_point = df[df['type_for_'+str(weeks)] == 'Actual Injuries'].tail(1)
    extra_point['type_for_'+str(weeks)] = 'Predicted Injuries'
    df = pd.concat([df, extra_point])

    df.loc[df['attacker'] == 1, 'position'] = 'attacker'
    df.loc[df['defender'] == 1, 'position'] = 'defender'
    df.loc[df['goalkeeper'] == 1, 'position'] = 'goalkeeper'
    df.loc[df['midfielder'] == 1, 'position'] = 'midfielder'

    df = df[['FBRefID', 'name', 'position', 'date', 'Min_cum', 'cum_week', 'cum_injury_prediction_in_'+str(weeks)+'_week', \
        'type_for_'+str(weeks)]]

    df = df.rename(columns={'type_for_'+str(weeks):'type_for_'+str(weeks)+'_weeks'})

    player_for_df_final = ['Lionel Messi', 'Cristiano Ronaldo', 'Neymar', 'Robert Lewandowski', 'Mohamed Salah', 'Kevin De Bruyne', 'Erling Haaland', \
    'Harry Kane', 'Virgil van Dijk', 'Eden Hazard', 'Karim Benzema', 'Paul Pogba', 'Sergio Ramos', 'Jadon Sancho', 'Romelu Lukaku', \
    'Joshua Kimmich', 'Antoine Griezmann', 'Toni Kroos', 'Jan Oblak', 'Manuel Neuer', 'Gareth Bale', 'Raheem Sterling', 'Frenkie de Jong', \
    'Thiago Silva', 'Paulo Dybala']

    df_final = pd.DataFrame([])

    for n in player_for_df_final:
        df_final = df_final.append(df[df['name'] == n])
    
    return df

In [193]:
test = clean_it(12, quarter_df)

In [73]:
def merge_it(list):
    df_final = pd.DataFrame([])

    for df in list:
        if df_final.empty:
            df_final = df
        else:
            df_final = pd.merge(df_final, df, left_on=['FBRefID', 'name', 'position', 'date', 'Min_cum', 'cum_week'], right_on=['FBRefID', \
                'name', 'position', 'date', 'Min_cum', 'cum_week'], how='outer')

    return df_final    

In [74]:
df_list = [clean_it(1, week_df), clean_it(4, month_df), clean_it(12, quarter_df), clean_it(26, semester_df), clean_it(52, year_df)]

In [75]:
df_for_tool = merge_it(df_list)

In [77]:
# Saving dataframe to parquet file

df_for_tool.to_parquet('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Files for Interactive Prediction Section/df_for_tool.parquet')

In [100]:
df_picked_player = df_for_tool[df_for_tool['name'] == 'Neymar']

In [101]:
def plot_it(weeks, df):
    domain_min = df['cum_injury_prediction_in_'+str(weeks)+'_week'].min()-5
    domain_max = df['cum_injury_prediction_in_'+str(weeks)+'_week'].max()+5

    chart = alt.Chart(df).mark_line().encode(x=alt.X('date', timeUnit='yearmonthdate', axis=alt.Axis(tickCount=4)), \
        y=alt.Y('cum_injury_prediction_in_'+str(weeks)+'_week', scale=alt.Scale(domain=[domain_min, domain_max])), \
        color=alt.Color('type_for_'+str(weeks)+'_weeks'), strokeDash=alt.condition(alt.datum.symbol == 'type_for_'+str(weeks)+'_weeks', \
        alt.value([5, 5]), alt.value([0]))).properties(title = str(df['name'].iloc[0])+"'s Injury Prediction for "+str(weeks)+" weeks", \
        width=800, height=300)
    return chart

In [104]:
df_picked_player

,FBRefID,name,position,date,Min_cum,cum_week,cum_injury_prediction_in_1_week,type_for_1_weeks,cum_injury_prediction_in_4_week,type_for_4_weeks,cum_injury_prediction_in_12_week,type_for_12_weeks,cum_injury_prediction_in_26_week,type_for_26_weeks,cum_injury_prediction_in_52_week,type_for_52_weeks
1857,69384e5d,Neymar,attacker,2013-06-23,73.0,0.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,9.0,Actual Injuries
1858,69384e5d,Neymar,attacker,2013-06-30,231.0,1.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,9.0,Actual Injuries
1859,69384e5d,Neymar,attacker,2013-07-07,410.0,2.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,9.0,Actual Injuries
1860,69384e5d,Neymar,attacker,2013-07-14,410.0,4.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,9.0,Actual Injuries
1861,69384e5d,Neymar,attacker,2013-07-21,410.0,5.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,9.0,Actual Injuries
1862,69384e5d,Neymar,attacker,2013-07-28,410.0,6.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,1.0,Actual Injuries,10.0,Actual Injuries
1863,69384e5d,Neymar,attacker,2013-08-04,410.0,7.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,2.0,Actual Injuries,11.0,Actual Injuries
1864,69384e5d,Neymar,attacker,2013-08-11,410.0,8.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,3.0,Actual Injuries,12.0,Actual Injuries
1865,69384e5d,Neymar,attacker,2013-08-18,410.0,9.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,4.0,Actual Injuries,13.0,Actual Injuries
1866,69384e5d,Neymar,attacker,2013-08-25,438.0,9.0,0.0,Actual Injuries,0.0,Actual Injuries,0.0,Actual Injuries,4.0,Actual Injuries,14.0,Actual Injuries


In [103]:
plot_it(26, df_picked_player)

alt.Chart(...)